In [61]:
import ast
import glob
import json
import re

import pandas as pd
import tensorflow as tf

from collections import Counter

/opt/anaconda3/envs/nasa/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88, got 80
  return f(*args, **kwds)
/opt/anaconda3/envs/nasa/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
  return f(*args, **kwds)


In [62]:
with open('/Users/xavierevans/nasa/notebooks/06-27-2022_ScienceKeywordClassificationDict.txt') as f:
    data = f.read()
data = data.replace("'", '"')
data = json.loads(data)

In [63]:
with open('/Users/xavierevans/nasa/notebooks/06-27-2022_DeepestKeywords.txt') as f:
    deepest_keywords = f.read()
deepest_keywords = deepest_keywords.replace('\n', '')
deepest_keywords = list(ast.literal_eval(deepest_keywords))
deepest_keywords[:5]

['Soil Porosity',
 'Particle Size Distribution',
 'Cloud-Cleared Infrared Radiance',
 'Land Use/Land Cover',
 'Sulfur Hexafluoride']

In [64]:
keywords = list(data.keys())
keyword_to_index_list = dict(zip(keywords, range(len(keywords))))
keywords[:5]

['Hydrogen Cyanide',
 'Isoprene',
 'Conduction',
 'Topography',
 'Cloud Microphysics']

In [65]:
keyword_to_index_list

{'Hydrogen Cyanide': 0,
 'Isoprene': 1,
 'Conduction': 2,
 'Topography': 3,
 'Cloud Microphysics': 4,
 'Incoming Solar Radiation': 5,
 'Cloud Fraction Vertical Distribution': 6,
 'Infrared Wavelengths': 7,
 'Barometric Altitude': 8,
 'Vorticity': 9,
 'Wind Speed Tendency': 10,
 'Surface Pressure': 11,
 'Climate Indicators': 12,
 'Ocean Heat Budget': 13,
 'Water Vapor Processes': 14,
 'Public Health': 15,
 'Clouds': 16,
 'Solar Energetic Particle Properties': 17,
 'Oceans': 18,
 'Volcanic Activity': 19,
 'Wind Dynamics': 20,
 'Absorption': 21,
 'Radio Wave': 22,
 'Ecosystem Functions': 23,
 'Sea Surface Height': 24,
 'Carbon': 25,
 'Upper Level Winds': 26,
 'Sublimation': 27,
 'Atmospheric Carbon Dioxide': 28,
 'Energy Deposition': 29,
 'Net Radiation': 30,
 'Antenna Temperature': 31,
 'Boundary Layer Temperature': 32,
 'Cloud Base Pressure': 33,
 'Heat Flux': 34,
 'Albedo': 35,
 'Chlorofluorocarbons': 36,
 'Chlorine Nitrate': 37,
 'Specific Humidity': 38,
 'Nitrate': 39,
 'Cloud Emissi

In [66]:
path = "/Users/xavierevans/nasa/metadata"
json_files = glob.glob(path + "/*.json")

In [67]:
json_keys = []
for file in json_files:
    with open(file) as f:
        data = json.load(f)
        json_keys += data.keys()
json_keys = list(set(json_keys))

In [68]:
json_values = []
for file in json_files:
    with open(file) as f:
        data = json.load(f)
        json_values.append([data[key] if key in data.keys()
                            else None for key in json_keys])

In [69]:
df = pd.DataFrame(json_values, columns=json_keys)

In [70]:
def clean_text(text):
    return text.replace('(', '').replace(')', '').replace('/', ' ').lower()

In [71]:
titles = df['EntryTitle'].to_list()
titles = [clean_text(title) for title in titles]
titles_text = ' '.join(titles)
titles_text[:1000]

'gridded monthly time-mean observation minus analysis oma values 0.5 x 0.667 degree v001 ma_hirs2_tirosn_oma at ges disc suomi npp cris level 1b full spectral resolution v2 snppcrisl1b at ges disc gpm ssm i on f13 gprof climate-based radiometer precipitation profiling l2 1.5 hours 12 km v05 gpm_2agproff13ssmi_clim at ges disc sentinel-5p tropomi radiance product band 7 swir detector l1b 5.5km x 7km v1 s5p_l1b_ra_bd7_hir at ges disc omi aura no2 tropospheric, stratospheric & total columns minds daily l2 global gridded 0.25 degree x 0.25 degree v1.1 omi_minds_no2g at ges disc mls aura level 3 monthly binned hydroxyl oh mixing ratio on assorted grids v004 ml3mboh at ges disc gridded monthly time-mean observation minus analysis oma values 0.5 x 0.667 degree v001 ma_ssu_noaa07_oma at ges disc suomi npp cris level 1b normal spectral resolution v1 snppcrisl1bnsr at ges disc modis aqua aerosol 5-min l2 swath subset 10km along mls v002 mam04s0 at ges disc gpm gmi antenna temperatures l1base 1.5

In [72]:
from nltk.tokenize import word_tokenize

In [73]:
token_counter = Counter(word_tokenize(titles_text)).most_common()

token2index = {'<OOV>': 0}
token2index.update({token: index for (token, _), index in zip(token_counter, range(1, len(token_counter) + 1))})
index2token = {value: key for key, value in token2index.items()}

tokens = list(token2index.keys())[1:]

In [74]:
with open('/Users/xavierevans/nasa/notebooks/06-29-2022_StopWords.txt') as f:
    stop_words = f.read().splitlines()

In [75]:
def tokenize(text):
    split_text = word_tokenize(clean_text(text))
    return [token2index[token] if token in tokens else token2index['<OOV>'] for token in split_text]

def tokenize_remove_stop_words(text):
    split_text = clean_text(text).split()
    split_text_without_stop_words = []
    for token in split_text:
        if token not in stop_words:
            split_text_without_stop_words.append(token)
    return [token2index[token] if token in tokens else token2index['<OOV>'] for token in split_text_without_stop_words]

def inverse_tokenize(tokenized_text):
    return ' '.join([index2token[index] for index in tokenized_text])

In [76]:
example_titles = titles[:5]
example_tokenized_titles = [tokenize(title) for title in example_titles]
example_titles

['gridded monthly time-mean observation minus analysis oma values 0.5 x 0.667 degree v001 ma_hirs2_tirosn_oma at ges disc',
 'suomi npp cris level 1b full spectral resolution v2 snppcrisl1b at ges disc',
 'gpm ssm i on f13 gprof climate-based radiometer precipitation profiling l2 1.5 hours 12 km v05 gpm_2agproff13ssmi_clim at ges disc',
 'sentinel-5p tropomi radiance product band 7 swir detector l1b 5.5km x 7km v1 s5p_l1b_ra_bd7_hir at ges disc',
 'omi aura no2 tropospheric, stratospheric & total columns minds daily l2 global gridded 0.25 degree x 0.25 degree v1.1 omi_minds_no2g at ges disc']

In [77]:
clean_title_to_original_title = dict(zip([clean_text(title) for title in titles], df['EntryTitle'].to_list()))

In [78]:
tokenized_titles = [tokenize(title) for title in titles]

In [79]:
def get_exact_match_score(tokenized_query):
    title_score_list = []
    for tokenized_title in tokenized_titles:
        match_count = 0
        for index in tokenized_query:
            match_count += 1 if index in tokenized_title else 0
        exact_match_score = match_count / len(tokenized_query)
        title_score_list.append(exact_match_score)
    return title_score_list

In [80]:
import operator

def get_best_exact_match_results(query):
    tokenized_query = tokenize_remove_stop_words(query)
    exact_match_scores = list(zip(df['EntryTitle'].to_list(), get_exact_match_score(tokenized_query)))
    exact_match_scores.sort(reverse=True, key=operator.itemgetter(1))
    return exact_match_scores

In [81]:
query = "UARS PEM Level 2 AXIS 1 V001 (UARPE2AXIS1)"
get_best_exact_match_results(query)

[('UARS PEM Level 2 AXIS 1 V001 (UARPE2AXIS1) at GES DISC', 1.0),
 ('UARS PEM Level 2 AXIS 2 V001 (UARPE2AXIS2) at GES DISC', 0.75),
 ('UARS PEM Level 2 VMAG AC V001 (UARPE2VMAGAC) at GES DISC', 0.625),
 ('UARS PEM Level 2 MEPS V001 (UARPE2MEPS) at GES DISC', 0.625),
 ('UARS PEM Level 2 HEPS B V001 (UARPE2HEPSB) at GES DISC', 0.625),
 ('UARS PEM Level 2 VMAG DC V001 (UARPE2VMAGDC) at GES DISC', 0.625),
 ('UARS PEM Level 2 HEPS A V002 (UARPE2HEPSA) at GES DISC', 0.5),
 ('MRIR/Nimbus-3 Level 1 Meteorological Radiation Data V001 (MRIRN3L1) at GES DISC',
  0.375),
 ('TIROS-4 Medium-Resolution Scanning Radiometer Level 1 Final Meteorological Radiation Data V001 (TIROS4L1FMRT) at GES DISC',
  0.375),
 ('TIROS-3 Medium-Resolution Scanning Radiometer Level 1 Final Meteorological Radiation Data V001 (TIROS3L1FMRT) at GES DISC',
  0.375),
 ('TIROS-2 Medium-Resolution Scanning Radiometer Level 1 Final Meteorological Radiation Data V001 (TIROS2L1FMRT) at GES DISC',
  0.375),
 ('SIRS/Nimbus-3 Level

We want to use embeddings of all of the words in the English language. This is provided to us with Glove. When the user inputs a query, we need to compare every single word of a query to every single word of each of the titles. So if there are five words in the query, we should iterate through those five words, get their embedding, and finds the titles which have a word in it that is most similar to each of the words. As we could imagine, it makes the most sense that we would want to return databases that contain all five of the words in the query. It doesn't really matter what order the words are in, because if every single word is contained in the title, there is a really good chance that the titles are similar, unless the query is extremely general. So, we can just iterate over the words in the title and choose the score for the most similar words. So for example, the word inquiry that we are considering in one given iteration might not be present in the title. The query might have the word black and the wording the title might be dark. However, these are pretty similar, and maybe it turns out that Dark is the closest word to black in the title of the database. So, we end up sticking with that score, which maybe turns out to be .1. Then we continue through the other databases and get the score, where the score might be even more optimal if the word is even more similar to black. Then, we will continue to iterate over the other words in the query. Maybe the query was originally black sky. Now we need to compare sky to every single word in every single title for all the databases, and again, the score will be more optimal if sky is actually contained in the title. But air would be pretty similar and so wood clouds, so maybe their embeddings would be quite similar anyways and the score would still be pretty good.

# Embeddings

In [82]:
with open('/Users/xavierevans/nasa/glove/glove.6B.50d.txt') as f:
    embedding_strings = f.read().splitlines()

In [83]:
def to_float_vector(string_vector):
    to_float = lambda x : float(x)
    return list(map(to_float, string_vector))

In [84]:
split_embedding_strings = [embedding_string.split() for embedding_string in embedding_strings]
split_embedding_strings_word_and_vector = [(split_embedding_string[0], to_float_vector(split_embedding_string[1:]))
                                           for split_embedding_string in split_embedding_strings]
embedding_dict = {word:vector for word, vector in split_embedding_strings_word_and_vector}

In [85]:
embedding_dict['mean'][:5]

[-0.067512, 0.29494, 0.20235, -0.59841, 0.43585]

In [86]:
from scipy import spatial

In [99]:
word_query = "abortion"
distances = [(word, spatial.distance.euclidean(embedding_dict[word], embedding_dict[word_query])) for word in embedding_dict.keys()]
distances.sort(key=operator.itemgetter(1))

In [100]:
distances

[('abortion', 0.0),
 ('gay', 3.4935173189104356),
 ('advocates', 3.509498429635637),
 ('affirmative', 3.695314498055206),
 ('opposes', 3.812397136843071),
 ('oppose', 3.8930654981589066),
 ('euthanasia', 3.9165706755564873),
 ('contraception', 3.9758488737424367),
 ('homosexuals', 3.9991875180688896),
 ('homosexuality', 4.013545930461428),
 ('arguing', 4.018673593889334),
 ('adoption', 4.033334024567255),
 ('opposed', 4.036107022032905),
 ('advocating', 4.043968096128232),
 ('gays', 4.050526233581138),
 ('banning', 4.1193063209124166),
 ('amendment', 4.1627384646247245),
 ('proponents', 4.196730255069684),
 ('repeal', 4.197572682641718),
 ('advocate', 4.222730954529311),
 ('favor', 4.2295805524785965),
 ('legislation', 4.24042666701088),
 ('favoring', 4.2581775564608435),
 ('legalization', 4.284124192322159),
 ('unconstitutional', 4.292582902406079),
 ('smoking', 4.317980043049761),
 ('legalizing', 4.352728852838958),
 ('lesbians', 4.393234194708495),
 ('advocated', 4.400565147104972),

In [107]:
class EmbeddingVector:
    def __init__(self, vector):
        self.vector = vector

    def get_closest(self):
        distances = [(word, spatial.distance.euclidean(embedding_dict[word], self.vector)) for word in embedding_dict.keys()]
        distances.sort(key=operator.itemgetter(1))
        return distances[1:]

In [109]:
word = 'mother'
e_word = EmbeddingVector(embedding_dict[word])
e_word.get_closest()

[('daughter', 1.8773749893108862),
 ('wife', 1.9073611868891667),
 ('grandmother', 2.0912483820519),
 ('husband', 2.384309630902054),
 ('sister', 2.5622836965187252),
 ('father', 2.5769109205832104),
 ('aunt', 2.7265028492765193),
 ('woman', 2.7886035127748845),
 ('friend', 2.8784852184657708),
 ('her', 2.88367326722936),
 ('daughters', 2.9695031568102523),
 ('herself', 3.042500037818117),
 ('she', 3.077548569026611),
 ('niece', 3.0800572365477317),
 ('parents', 3.1379056262597897),
 ('granddaughter', 3.2041372971165023),
 ('son', 3.2280756764078573),
 ('girl', 3.2485381793263644),
 ('girlfriend', 3.2704074269043315),
 ('widow', 3.277406054273652),
 ('boy', 3.303997625569039),
 ('child', 3.3123212536031357),
 ('lover', 3.3840602053895696),
 ('cousin', 3.4062027628196248),
 ('grandfather', 3.4441002865275943),
 ('couple', 3.48616885397352),
 ('married', 3.548815815324197),
 ('dying', 3.5577600964965734),
 ('brother', 3.58613904395011),
 ('uncle', 3.6011856025742475),
 ('friends', 3.6393

In [111]:
df.columns

Index(['ShortName', 'DataLanguage', 'ArchiveAndDistributionInformation',
       'LocationKeywords', 'ContactGroups', 'AccessConstraints', 'RelatedUrls',
       'AncillaryKeywords', 'Platforms', 'DirectoryNames',
       'CollectionProgress', 'Abstract', 'DOI', 'MetadataDates', 'Projects',
       'ContactPersons', 'Quality', 'Purpose', 'SpatialExtent',
       'ProcessingLevel', 'DirectDistributionInformation', 'DataDates',
       'CollectionCitations', 'TemporalExtents', 'ISOTopicCategories',
       'DataCenters', 'Version', 'PublicationReferences', 'ScienceKeywords',
       'VersionDescription', 'UseConstraints', 'MetadataAssociations',
       'EntryTitle', 'CollectionDataType'],
      dtype='object')

In [117]:
for elt in df['Abstract'].to_list():
    print(elt)

The differences between the observations and the forecast background used for the analysis (the innovations or O-F for short) and those between the observations and the final analysis (O-A) are by-products of any assimilation system and provide information about the quality of the analysis and the impact of the observations. Innovations have been traditionally used to diagnose observation, background and analysis errors at observation locations (Hollingsworth and Lonnberg 1989; Dee and da Silva 1999). At the most simplistic level, innovation variances can be used as an upper bound on background errors, which are, in turn, an upper bound on the analysis errors. With more processing (and the assumption of optimality), the O-F and O-A statistics can be used to estimate observation, background and analysis errors (Desroziers et al. 2005). They can also be used to estimate the systematic and random errors in the analysis fields. Unfortunately, such data are usually not readily available wit